In [24]:
# Install requirements, uncomment to run
# !pip install geopandas pillow label-studio-converter

# Uploading Savmap dataset to label studio

In [1]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path
import numpy as np
import json
import pandas as pd
from torchvision.ops import nms
import torch
from label_studio_sdk import Client
from dotenv import load_dotenv
from label_studio_tools.core.utils.io import get_local_path
from urllib.parse import unquote

In [14]:
#  load data
root = r"D:\savmap_dataset_v2"
polygons = gdp.read_file(os.path.join(root,'savmap_annotations_2014.geojson'))

path_to_images = os.path.join(root,'images')
path_to_labels = os.path.join(root,'labels')

In [57]:
# polygon points
xx,yy = polygons['geometry'].iloc[0].exterior.coords.xy
np.array(list(zip(xx,yy))) * 100 / np.array([4000,3000]).reshape((1,2))

In [54]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in tqdm(range(len(polygons))):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = os.path.join(path_to_images,f"{polygons['IMAGEUUID'].iloc[i]}.JPG")
    # try:
    width, height = Image.open(image_path).size
    # except:
    #     continue
    polygons['x_min'].iat[i] = max(int(x_min),0)
    polygons['x_max'].iat[i] = max(int(x_max),0)
    polygons['y_min'].iat[i] = max(int(y_min),0)
    polygons['y_max'].iat[i] = max(int(y_max),0)
    polygons['x'].iat[i] = 0.5*(x_max+x_min)
    polygons['y'].iat[i] = 0.5*(y_max+y_min)
    polygons['width'].iat[i] = width
    polygons['height'].iat[i] = height

polygons['bbox_w'] = (polygons['x_max'] - polygons['x_min'])*1.0
polygons['bbox_h'] = (polygons['y_max'] - polygons['y_min'])*1.0
polygons['class'] = 0

100%|██████████| 7474/7474 [00:05<00:00, 1286.68it/s]


In [55]:
# non-max suppresion
# https://pytorch.org/vision/stable/generated/torchvision.ops.nms.html

def nms_to_bbox(df_annotations:pd.DataFrame,iou_threshold:float=0.5):
    dfs = list()

    for IMAGEUUID, df in tqdm(df_annotations.groupby('IMAGEUUID')):

        df.reset_index(inplace=True)

        bbox = df[['x_min','y_min','x_max','y_max']].to_numpy().astype(float)
        scores = np.ones((bbox.shape[0],)).astype(float)

        bbox_indices= nms(boxes=torch.from_numpy(bbox),
                            scores=torch.from_numpy(scores),
                            iou_threshold=iou_threshold).numpy()
        
        # print(bbox[bbox_indices],'\n',df.iloc[bbox_indices,:])

        dfs.append(df.iloc[bbox_indices,:].copy())

    return pd.concat(dfs,axis=0).reset_index()

In [56]:
# nms filtering of bbox
df_filtered = nms_to_bbox(polygons,iou_threshold=0.4)
df_filtered.rename(columns={'class':'labels'},inplace=True)

  0%|          | 0/654 [00:00<?, ?it/s]

100%|██████████| 654/654 [00:01<00:00, 505.18it/s]


In [57]:
df_filtered.drop(columns=['level_0','index'],inplace=True)

In [58]:
# filtered df vs unfiltered df
len(df_filtered), len(polygons)

(3313, 7474)

In [59]:
df_filtered.head(1)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,labels
0,003a34ee6b7841e6851b8fe511ebe102,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",1503,962,1537,996,4000,3000,1520.0,979.0,34.0,34.0,0


In [60]:
# save filtered annotations as csv
savepath = os.path.join(root,'savmap_annotations_2014_filtered.csv')
df_filtered['images'] = df_filtered['IMAGEUUID'].apply(lambda x: f"{x}.JPG")
df_filtered.to_csv(savepath,index=False)

In [61]:
savepath = os.path.join(root,'savmap_annotations_2014_filtered.csv')
df_filtered = pd.read_csv(savepath)
df_filtered.head(2)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,labels,images
0,003a34ee6b7841e6851b8fe511ebe102,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",1503,962,1537,996,4000,3000,1520.0,979.0,34.0,34.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG
1,003a34ee6b7841e6851b8fe511ebe102,ece6f29e46544b4eaeb4c057c0ecfe06,"POLYGON ((1688 858, 1695 865, 1694 870, 1692 8...",1679,858,1695,875,4000,3000,1687.0,866.5,16.0,17.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG


In [62]:
df_filtered[df_filtered['IMAGEUUID'] == '003a34ee6b7841e6851b8fe511ebe102']

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,labels,images
0,003a34ee6b7841e6851b8fe511ebe102,d57402df14a84b58a411bade4f9d14f1,"POLYGON ((1514 962, 1524 970, 1536 968, 1536 9...",1503,962,1537,996,4000,3000,1520.0,979.0,34.0,34.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG
1,003a34ee6b7841e6851b8fe511ebe102,ece6f29e46544b4eaeb4c057c0ecfe06,"POLYGON ((1688 858, 1695 865, 1694 870, 1692 8...",1679,858,1695,875,4000,3000,1687.0,866.5,16.0,17.0,0,003a34ee6b7841e6851b8fe511ebe102.JPG


In [ ]:
# Tiling images
# !python ../HerdNet/tools/patcher.py "D:\savmap_dataset_v2\images" 2000 2000 100 -dest "D:\savmap_dataset_v2\images_splits" -csv "D:\savmap_dataset_v2\savmap_annotations_2014_filtered.csv" -min 0.5 -all False

In [ ]:
# load groundtruth for split images
df_splits = pd.read_csv(r"D:\savmap_dataset_v2\images_splits\gt.csv")
df_splits["IMAGEUUID"] = df_splits['images'].apply(lambda x: str(x).split(".")[0])
df_splits['x'] = 0.5*(df_splits["x_min"]+df_splits["x_max"])
df_splits['y'] = 0.5*(df_splits["y_min"]+df_splits["y_max"])
df_splits["width"] = 0.
df_splits["height"] = 0.
df_splits['bbox_w'] = df_splits['x_max'] - df_splits['x_min']
df_splits['bbox_h'] = df_splits['y_max'] - df_splits['y_min']
for name,df in tqdm(df_splits.groupby("images")):
    image_path = os.path.join(r"D:\savmap_dataset_v2\images_splits",name)
    width, height = Image.open(image_path).size
    mask = (df_splits["images"] == name)
    df_splits.loc[mask,'width']  = width
    df_splits.loc[mask,'height'] = height

100%|██████████| 1161/1161 [00:01<00:00, 853.69it/s]


In [3]:
df_splits.head(3)

,images,labels,base_images,x_min,y_min,x_max,y_max,IMAGEUUID,x,y,width,height,bbox_w,bbox_h
0,003a34ee6b7841e6851b8fe511ebe102_0.JPG,0,003a34ee6b7841e6851b8fe511ebe102.JPG,1503,962,1537,996,003a34ee6b7841e6851b8fe511ebe102_0,1520.0,979.0,2000.0,2000.0,34.0,34.0
1,003a34ee6b7841e6851b8fe511ebe102_0.JPG,0,003a34ee6b7841e6851b8fe511ebe102.JPG,1679,858,1695,875,003a34ee6b7841e6851b8fe511ebe102_0,1687.0,866.5,2000.0,2000.0,16.0,17.0
2,0078d29a8d0b489caa3425969c7477ac_0.JPG,0,0078d29a8d0b489caa3425969c7477ac.JPG,620,1653,677,1689,0078d29a8d0b489caa3425969c7477ac_0,648.5,1671.0,2000.0,2000.0,57.0,36.0


In [ ]:
# Select labels
# data = polygons.loc[:,['x_min','x_max','y_min','y_max','IMAGEUUID','width','height','class']].copy()
# data.rename(columns={'IMAGEUUID':'filename',
#                      'x_max':'xmax',
#                      'x_min':'xmin',
#                      'y_min':'ymin',
#                      'y_max':'ymax'},inplace=True)
# data['filename'] = data['filename'].apply(lambda x: f"{x}.JPG")
# data = data.dropna()
# data = data[['filename','class','width', 'height','xmin','ymin','xmax','ymax']]
# data.head(1)


In [ ]:
# # Convert CSV to COCO
# save_json_path = ... #'cocoformat.json'

# images = []
# categories = []
# annotations = []

# category = {}
# category["supercategory"] = 'None'
# category["id"] = 0
# category["category_name"] = 'wildlife'
# categories.append(category)

# data['fileid'] = data['filename'].astype('category').cat.codes
# data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
# # data['categoryid'] = data['categoryid']+1
# data['annid'] = data.index

# def image(row):
#     image = {}
#     image["height"] = row.height
#     image["width"] = row.width
#     image["id"] = row.fileid
#     image["file_name"] = row.filename
#     return image

# # def category(row):
# #     category = {}
# #     category["supercategory"] = 'None'
# #     category["id"] = row.categoryid
# #     category["category_name"] = row[2]
# #     return category

# def annotation(row):
#     annotation = {}
#     area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
#     annotation["segmentation"] = []
#     annotation["iscrowd"] = 0
#     annotation["area"] = area
#     annotation["image_id"] = row.fileid
#     annotation["score"] = 1.0

#     annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

#     annotation["category_id"] = row.categoryid
#     annotation["id"] = row.annid
#     return annotation

# for row in data.itertuples():
#     annotations.append(annotation(row))

# imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
# for row in imagedf.itertuples():
#     images.append(image(row))

# # catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
# # for row in catdf.itertuples():
# #     categories.append(category(row))

# data_coco = {}
# data_coco["images"] = images
# data_coco["categories"] = categories
# data_coco["annotations"] = annotations
# # json.dump(data_coco, open(save_json_path, "w"), indent=4)

In [ ]:
# groundtruth = {annot['file_name']:[] for annot in data_coco['images']}
# for annot,image_data in zip(data_coco['annotations'],data_coco['images']):
#     annot.update(image_data)
#     # pprint.pp(annot)
#     annot['category_name'] = category['category_name']
#     groundtruth[annot['file_name']].append(annot)

In [5]:
dotenv_path = r"..\.env"
if dotenv_path is not None:
    load_dotenv(dotenv_path=dotenv_path)

In [6]:
# Connect to the Label Studio API and check the connection
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL')
API_KEY = os.getenv("LABEL_STUDIO_API_KEY")
labelstudio_client = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

In [ ]:
# url = "/data/local-files/?d=Users%5Cfadel%5COneDrive%5CBureau%5CWILD-AI%5Cdatalabeling%5Cdata%5Ctrain_wildai%5Cimages%5C003a34ee6b7841e6851b8fe511ebe102.JPG"

In [ ]:
# unquote(url)

In [82]:
# Uploading Polygons
def format_prediction_polygon(points:list,width:int,height:int,label:list=['wildlife',]):
    
    template = {
            "original_width": width,
            "original_height": height,
            "image_rotation": 0,
            "value": {
                "points": points,
                "closed":True,
                "polygonlabels": label
            }
    }

    return template

def format_predictions_polygon(polygons:list,widths:list,heights:list,label:list=['wildlife',]):

    results = list()
    for polygon,w,h in zip(polygons,widths,heights):
        xx,yy=polygon.exterior.coords.xy
        points = list(zip(xx,yy))

        # convert points to percent
        points = np.array(points) * 100 / np.array([w,h]).reshape((1,2))
        points = points.tolist()      

        # append result
        results.append(format_prediction_polygon(points,width=w,height=h,label=label))

    return results


# uploading polygons
 # Select project
# project = labelstudio_client.get_project(id=project_id)
# # Upload predictions for each task
# tasks = project.get_tasks()
# if top_n > 0:
#     tasks = sorted(tasks,key=lambda x:x['id'])[:top_n]

for task in tqdm(tasks[:10],desc="Uploading predictions"):
    task_id = task['id']
    img_url = unquote(task['data']['image'])
    img_path = get_local_path(img_url)
    img_name = Path(img_path).stem
    mask = polygons['IMAGEUUID']==img_name
    polys = polygons.loc[mask,['IMAGEUUID','geometry','width','height']]
    formatted_pred = format_predictions_polygon(polygons=polys['geometry'],widths=polys['width'],heights=polys['height'],label=['wildlife',])
    # conf_scores = [pred['score'] for pred in prediction]
    # max_score = 0.0
    project.create_prediction(task_id=task_id,
                               result=formatted_pred,model_version='gt')

In [64]:
import traceback

# uploading gt bbox
def format_prediction_bbox(pred:dict,img_height:int,img_width:int,
                           from_name:str='label',
                           to_name:str='image',
                           label_type:str='rectanglelabels') -> dict:
        """Converts prediction Label studio format

        Args:
            pred (dict): prediction in coco format
            img_height (int): image height
            img_width (int): image width

        Returns:
            dict: Label studio formated prediction
        """
        # formatting the prediction to work with Label studio
        x, y, width, height = pred['bbox']
        label = pred['category_name']
        # score = pred['score']
        # if not isinstance(score,float):
        #     score = 0.0
        template = {
                    "from_name": from_name,
                    "to_name": to_name,
                    "type": label_type,
                    "original_width":img_width,
                    "original_height":img_height,
                    "image_rotation":0,
                    'value': {
                        label_type: [label,],
                        'x': x / img_width * 100.,
                        'y': y / img_height * 100.,
                        'width': width / img_width * 100.,
                        'height': height / img_height * 100.,
                        'rotation':0.
                    },
                    # 'score': score
        }
        return template

def format_predictions_bbox(xs:list,ys:list,ws:list,hs:list,img_width:int,img_height:int,label:str='wildlife'):
      
    results = list()
    for x,y,w,h in zip(xs,ys,ws,hs):
        
        annot = {'bbox':[x,y,w,h],
                 'category_name':label,
                 'score':None
                 }

        # append result
        results.append(format_prediction_bbox(annot,img_width=img_width,img_height=img_height))

    return results

def upload_predictions_bbox(df_annotations:pd.DataFrame,project_id:int):

    # get tasks
    project = labelstudio_client.get_project(id=project_id)
    tasks = project.get_tasks()

    # upload
    failed_uploads = set()
    for task in tqdm(tasks[:],desc="Uploading predictions"):
        task_id = task['id']
        img_url = unquote(task['data']['image'])
        img_path = get_local_path(img_url)
        img_name = Path(img_path).stem
        try:
            mask = df_annotations['IMAGEUUID']==img_name
            df_bbox = df_annotations.loc[mask,['x','y','bbox_w','bbox_h','width','height']]
            img_width = df_bbox['width'].iat[0]
            img_height = df_bbox['height'].iat[0]
            formatted_pred = format_predictions_bbox(xs=df_bbox['x'],
                                                    ys=df_bbox['y'],
                                                    ws=df_bbox['bbox_w'],
                                                    hs=df_bbox['bbox_h'],
                                                    img_width=int(img_width),
                                                    img_height=int(img_height),
                                                    label='wildlife'
                                                    )
            # project.create_annotation(task_id=task_id,
            #                         result=formatted_pred,
            #                         # model_version='gt'
            #                         )
            project.create_prediction(task_id=task_id,
                                    result=formatted_pred,
                                    model_version='gt')
        except Exception as e:
            failed_uploads.add(img_name)
            #  print("Failed for: ",img_name,e,end="\n")
            traceback.print_exc()
            break
    return failed_uploads

def delete_tasks(project_id:int,failed_uploads:set):

    # get tasks
    project = labelstudio_client.get_project(id=project_id)
    tasks = project.get_tasks()
    # delete tasks
    for task in tqdm(tasks[:],desc="deleting tasks"):
        task_id = task['id']
        img_url = unquote(task['data']['image'])
        img_path = get_local_path(img_url)
        img_name = Path(img_path).stem
        if img_name in failed_uploads:
            project.delete_task(task_id=task_id)

In [65]:
# savmap original
project_id = 1
failed_uploads = upload_predictions_bbox(df_annotations=df_filtered,project_id=project_id)

Uploading predictions:   0%|          | 0/654 [00:00<?, ?it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


D:\savmap_dataset_v2\images\003a34ee6b7841e6851b8fe511ebe102.JPG


Uploading predictions:   0%|          | 1/654 [00:00<02:20,  4.64it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


D:\savmap_dataset_v2\images\0078d29a8d0b489caa3425969c7477ac.JPG


Uploading predictions:   0%|          | 2/654 [00:00<02:23,  4.53it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


...

Uploading predictions: 100%|█████████▉| 652/654 [03:18<00:00,  3.27it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


D:\savmap_dataset_v2\images\ff24914832c2450ab5ee4bea75c540aa.JPG


Uploading predictions: 100%|██████████| 654/654 [03:18<00:00,  3.88it/s]

D:\savmap_dataset_v2\images\ffa17349e4154d1f86d632973e27d8a6.JPG


Uploading predictions: 100%|██████████| 654/654 [03:18<00:00,  3.29it/s]


In [50]:
len(failed_uploads)

654

In [ ]:
# savmap splits
project_id = 3
failed_uploads = upload_predictions_bbox(df_annotations=df_splits,project_id=project_id)

Uploading predictions:   0%|          | 0/1161 [00:00<?, ?it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


D:\savmap_dataset_v2\images_splits\003a34ee6b7841e6851b8fe511ebe102_0.JPG


Uploading predictions:   0%|          | 1/1161 [00:00<04:01,  4.80it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


D:\savmap_dataset_v2\images_splits\0078d29a8d0b489caa3425969c7477ac_0.JPG


Uploading predictions:   0%|          | 2/1161 [00:00<05:24,  3.57it/s]Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


...

Uploading predictions: 100%|█████████▉| 1160/1161 [04:37<00:00,  3.99it/s]

D:\savmap_dataset_v2\images_splits\ff24914832c2450ab5ee4bea75c540aa_4.JPG


Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.
Uploading predictions: 100%|██████████| 1161/1161 [04:38<00:00,  4.31it/s]

D:\savmap_dataset_v2\images_splits\ffa17349e4154d1f86d632973e27d8a6_4.JPG


Uploading predictions: 100%|██████████| 1161/1161 [04:38<00:00,  4.17it/s]


In [66]:
len(failed_uploads), '2da691c095e64729811fe7fe9c2241e9_5' in failed_uploads

(2764, True)

In [ ]:
# delete empty images
delete_tasks(df_annotations=df_splits,project_id=project_id,failed_uploads=failed_uploads)

In [79]:
# format_predictions(polygons=polys['geometry'],widths=polys['width'],heights=polys['height'],label=['wildlife',])

# Formatting savmap dataset
Formatting the savmap dataset so it can be imported in label studio through a json file

In [84]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path

In [92]:
#  load data
root = r"D:\savmap_dataset_v2"
polygons = gdp.read_file(os.path.join(root,'savmap_annotations_2014.geojson'))

# create directories
path_to_images = os.path.join(root,'images')
path_to_labels = os.path.join(root,'labels')
Path(path_to_images).mkdir(exist_ok=True,parents=True)
Path(path_to_labels).mkdir(exist_ok=True,parents=True)

In [3]:
# Move files
# ! mv ../data/savmap_dataset_v2/*.JPG ../data/savmap_dataset_v2/images

In [93]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in range(len(polygons)):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = os.path.join(path_to_images,f"{polygons['IMAGEUUID'].iloc[i]}.JPG")
    width, height = Image.open(image_path).size
    polygons['x_min'].iat[i] = max(0,int(x_min))
    polygons['x_max'].iat[i] = max(0,int(x_max))
    polygons['y_min'].iat[i] = max(0,int(y_min))
    polygons['y_max'].iat[i] = max(0,int(y_max))
    polygons['x'].iat[i] = 0.5*(x_max+x_min)
    polygons['y'].iat[i] = 0.5*(y_max+y_min)
    polygons['width'].iat[i] = width
    polygons['height'].iat[i] = height

# creat bbox width and height
polygons['bbox_w'] = polygons['x_max'] - polygons['x_min']
polygons['bbox_h'] = polygons['y_max'] - polygons['y_min']
polygons['class'] = 0

In [ ]:
polygons.head(1)

In [94]:
# convert to yolo format
def save_df_as_yolo(df_annotation:gdp.GeoDataFrame,dest_path_labels:str):
    
    cols = ['class','x','y','bbox_w','bbox_h']
    for col in cols:
        assert df_annotation[col].isna().sum()<1,'there are NaN values. Check out.'
        # df_annotation[col] = df_annotation[col].apply(int)

    # normalize values
    df_annotation.loc[:,'x'] = df_annotation['x']/df_annotation['width']
    df_annotation.loc[:,'y'] = df_annotation['y']/df_annotation['height']
    df_annotation.loc[:,'bbox_w'] = df_annotation['bbox_w']/df_annotation['width']
    df_annotation.loc[:,'bbox_h'] = df_annotation['bbox_h']/df_annotation['height']
    
    for image_name,df in tqdm(df_annotation.groupby('IMAGEUUID'),desc='Saving yolo labels'):
        txt_file = f'{image_name}.txt'
        df[cols].to_csv(os.path.join(dest_path_labels,txt_file),sep=' ',index=False,header=False)

In [95]:
save_df_as_yolo(df_annotation=polygons,dest_path_labels=path_to_labels)

Saving yolo labels:   0%|          | 0/654 [00:00<?, ?it/s]

Saving yolo labels: 100%|██████████| 654/654 [00:01<00:00, 463.09it/s]


In [ ]:
# tutorial here: https://github.com/HumanSignal/label-studio-converter/tree/master
# if it does not work, use a terminal
# !label-studio-converter import yolo -i "C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai" --image-ext ".JPG" --out-type "predictions" -o "C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\ls_tasks.json"

# Formatting General Dataset from Delplanque 2021

In [9]:
import pandas as pd
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm


## Tiling

In [ ]:
# Tiling test
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\test" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\tiled_data\test_tiled\images" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\test_big_size_A_B_E_K_WH_WB.csv"

In [ ]:
# Tiling val
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\val" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\val_tiled" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\val_big_size_A_B_E_K_WH_WB.csv"

In [ ]:
# Tiling Train
# !python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Extra training data\general_dataset\train" 640 640 64  -dest "D:\PhD\Data per camp\Extra training data\general_dataset\train_tiled" -pattern "**/*.JPG" -csv "D:\PhD\Data per camp\Extra training data\general_dataset\groundtruth\csv\train_big_size_A_B_E_K_WH_WB.csv"

## Converting to YOLO format

In [56]:
# convert to yolo format
def save_df_as_yolo(df_annotation:pd.DataFrame,dest_path_labels:str,is_detector:bool=False):
    
    cols = ['class','x','y','bbox_w','bbox_h']
    for col in cols:
        assert df_annotation[col].isna().sum()<1,'there are NaN values. Check out.'
        # df_annotation[col] = df_annotation[col].apply(int)
    
    for col in ['x','y','bbox_w','bbox_h']:
        df_annotation[col] = df_annotation[col].astype(float)

    # normalize values
    df_annotation.loc[:,'x'] = df_annotation['x']/df_annotation['width']
    df_annotation.loc[:,'y'] = df_annotation['y']/df_annotation['height']
    df_annotation.loc[:,'bbox_w'] = df_annotation['bbox_w']/df_annotation['width']
    df_annotation.loc[:,'bbox_h'] = df_annotation['bbox_h']/df_annotation['height']

    if is_detector:
        df_annotation['class'] = 0
    
    for image_name,df in tqdm(df_annotation.groupby('images'),desc='Saving yolo labels'):
        txt_file = f'{Path(image_name).stem}.txt'
        df[cols].to_csv(os.path.join(dest_path_labels,txt_file),sep=' ',index=False,header=False)

In [67]:
split='train' # 'train' 'test', 'val
root = Path(rf"D:\PhD\Data per camp\Extra training data\general_dataset\tiled_data\{split}_tiled")
path_to_csv = root/"gt.csv"
path_images = root/'images'
path_to_labels = root/'labels'
detection_mode=False # save label for wildlife detection only

In [68]:
df_annotations = pd.read_csv(path_to_csv)
# df_annotations.head(2)

# update df_annotations
df_annotations['width'] = 0.0
df_annotations['height'] = 0.0


for name in set(df_annotations.images):
    width, height = Image.open(root/f"images/{name}").size
    df_annotations.loc[df_annotations.images==name,'width'] = float(width)
    df_annotations.loc[df_annotations.images==name,'height'] = float(height)

df_annotations['x'] = 0.5*(df_annotations['x_min'] + df_annotations['x_max'])
df_annotations['y'] = 0.5*(df_annotations['y_min'] + df_annotations['y_max'])
df_annotations['bbox_h'] = df_annotations['y_max'] - df_annotations['y_min']
df_annotations['bbox_w'] = df_annotations['x_max'] - df_annotations['x_min']

df_annotations.rename(columns={'labels':'class'},inplace=True)

In [69]:
# df_annotations.head(2)
# df_annotations['class'].plot(kind='hist')

In [ ]:
save_df_as_yolo(df_annotations,path_to_labels,is_detector=detection_mode)

# WAID 

## Trasforming labels for detector training

In [17]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [ ]:
path = r"C:\Users\Machine Learning\Desktop\workspace-wildAI\datalabeling\data\WAID\labels"
path = Path(path)

for p in tqdm(path.glob("*/**/*.txt")):
    df = pd.read_csv(p,sep=" ",header=None)
    df.columns = ["class",'x','y','w','h']
    df['class'] = 0
    # un-comment to run
    # df.to_csv(p, sep=" ", header=False, index=False)

# Hard sample mining

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")

from datalabeling.annotator import Detector
from datalabeling.dataset.sampling import (get_preds_targets, 
                                           select_hard_samples,
                                           compute_detector_performance)
import pandas as pd
import yaml
import os

In [ ]:
# Define detector
# to speed up inference on intel, make
model = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                confidence_threshold=0.1,
                overlap_ratio=0.1,
                tilesize=1280,
                device='CPU',
                use_sliding_window=True,
                is_yolo_obb=True)

11/17/2024 13:25:24 - INFO - datalabeling.annotator.models -   Computing device: CPU


Loading C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model for OpenVINO inference...
Available openvino devices are: ['CPU', 'GPU', 'NPU']. Using device=AUTO:NPU,GPU,CPU.
Using OpenVINO LATENCY mode for inference...


In [4]:
# load groundtruth
with open(r"..\data\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)

train_images_path = [os.path.join(yolo_config['path'],yolo_config['train'][i]) for i in range(len(yolo_config['train']))]
train_labels_path = [p.replace('images','labels') for p in train_images_path]

In [5]:
train_images_path,train_labels_path

(['D:\\savmap_dataset_v2\\images'], ['D:\\savmap_dataset_v2\\labels'])

In [ ]:
df_results, df_labels, col_names = get_preds_targets(images_dirs=train_images_path,
                                                  pred_results_dir=r"..\data",
                                                  detector=model,
                                                  load_results=False)

In [7]:
df_labels.head(2)

,category_id,x1,y1,x2,y2,x3,y3,x4,y4,image_path,width,height
0,0,1503.0,962.0,1537.0,962.0,1537.0,996.0,1503.0,996.0,D:\savmap_dataset_v2\images\003a34ee6b7841e685...,4000,3000
1,0,1679.0,858.0,1695.0,858.0,1695.0,875.0,1679.0,875.0,D:\savmap_dataset_v2\images\003a34ee6b7841e685...,4000,3000


In [8]:
df_results.head(2)

,score,category_id,category_name,area,image_path,x_min,y_min,bbox_w,bbox_h,x_max,y_max
0,0.750977,0.0,wildlife,3889.0,D:\savmap_dataset_v2\images\003a34ee6b7841e685...,1357.804779,1883.299133,64.390442,60.401733,1422.195221,1943.700867
1,0.668457,0.0,wildlife,2527.0,D:\savmap_dataset_v2\images\003a34ee6b7841e685...,3635.426636,1375.409607,49.146729,51.430817,3684.573364,1426.840424


In [9]:
col_names

['category_id', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

In [10]:
df_results_per_img = compute_detector_performance(df_results,df_labels,col_names)
df_results_per_img.head(2)

100%|██████████| 653/653 [00:11<00:00, 54.74it/s]


,map50,map75,max_scores,image_paths
0,0.0,0.0,0.750977,D:\savmap_dataset_v2\images\003a34ee6b7841e685...
1,0.0,0.0,0.799805,D:\savmap_dataset_v2\images\0078d29a8d0b489caa...


In [11]:
# save hard samples
df_hard_negatives = select_hard_samples(df_results_per_img=df_results_per_img,
                                            map_thrs=0.3,
                                            score_thrs=0.7,
                                            save_path_samples=r"D:\hard_negative_samples\hard_samples.txt",
                                            root='D:\\',
                                            save_data_yaml=r"..\data\hard_samples.yaml"
                                            )

# Converting datasets to yolo<>OBB

In [63]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

def convert_yolo_to_obb(yolo_dataset_dir:str,output_dir:str)->None:

    cols = ['id','x1','y1','x2','y2','x3','y3','x4','y4']
    names = ['id','x','y','w','h']

    # Iterate through labels
    for label_path in tqdm(Path(yolo_dataset_dir).glob("*.txt"),desc='yolo->obb'):
        df = pd.read_csv(label_path,sep=' ',names=names)

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        for col in names[1:]:
            df[col] = df[col].astype(float)
        df['id'] = df['id'].astype(int)

        df['w'] = 0.5*df['w']
        df['h'] = 0.5*df['h']
        # top left
        df['x1'] = df['x'] - df['w']
        df['y1'] = df['y'] - df['h']
        # top right
        df['x2'] = df['x'] + df['w']
        df['y2'] = df['y'] - df['h']
        # bottom right
        df['x3'] = df['x'] + df['w']
        df['y3'] = df['y'] + df['h']
        # bottom left
        df['x4'] = df['x'] - df['w']
        df['y4'] = df['y'] + df['h']

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # save file
        df[cols].to_csv(Path(output_dir)/label_path.name,
                        sep=' ',index=False,header=False)

def convert_obb_to_yolo(obb_dataset_dir:str,output_dir:str)->None:

    names = ['id','x1','y1','x2','y2','x3','y3','x4','y4']
    cols = ['id','x','y','w','h']

    # Iterate through labels
    for label_path in tqdm(Path(obb_dataset_dir).glob("*.txt"),desc='obb->yolo'):
        df = pd.read_csv(label_path,sep=' ',names=names)

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # center
        df['x'] = (df['x1'] + df['x2'])/2.
        df['y'] = (df['y1'] + df['y4'])/2.
        # width
        df['w'] = df['x2'] - df['x1']
        # height
        df['h'] = df['y4'] - df['y1']

        # check bounds
        assert df[names[1:]].all().max() <=1., "max value <= 1"
        assert df[names[1:]].all().min() >= 0., "min value >=0"

        # save file
        df[cols].to_csv(Path(output_dir)/label_path.name,sep=' ',index=False,header=False)

In [ ]:
#uncomment to run
# convert_yolo_to_obb(yolo_dataset_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels",
#                     output_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels")

In [ ]:
# uncomment to run
# convert_obb_to_yolo(obb_dataset_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels",
#                     output_dir=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\labels")